In [131]:
import nltk
from collections import defaultdict
import pandas as  pd
import numpy as np
import time
import math



## Separating Training and Testing Data from Brown Corpus

In [52]:
brown_data = nltk.corpus.brown.tagged_words(tagset='universal')
brown_sents = nltk.corpus.brown.tagged_sents(tagset='universal')
# 70 % train dataset and 30% test dataset
size = int(len(brown_sents)*0.7)
train_brown = brown_sents[:size]
test_brown = brown_sents[size:]

In [53]:
brown_sents

[[('The', 'DET'), ('Fulton', 'NOUN'), ('County', 'NOUN'), ('Grand', 'ADJ'), ('Jury', 'NOUN'), ('said', 'VERB'), ('Friday', 'NOUN'), ('an', 'DET'), ('investigation', 'NOUN'), ('of', 'ADP'), ("Atlanta's", 'NOUN'), ('recent', 'ADJ'), ('primary', 'NOUN'), ('election', 'NOUN'), ('produced', 'VERB'), ('``', '.'), ('no', 'DET'), ('evidence', 'NOUN'), ("''", '.'), ('that', 'ADP'), ('any', 'DET'), ('irregularities', 'NOUN'), ('took', 'VERB'), ('place', 'NOUN'), ('.', '.')], [('The', 'DET'), ('jury', 'NOUN'), ('further', 'ADV'), ('said', 'VERB'), ('in', 'ADP'), ('term-end', 'NOUN'), ('presentments', 'NOUN'), ('that', 'ADP'), ('the', 'DET'), ('City', 'NOUN'), ('Executive', 'ADJ'), ('Committee', 'NOUN'), (',', '.'), ('which', 'DET'), ('had', 'VERB'), ('over-all', 'ADJ'), ('charge', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('election', 'NOUN'), (',', '.'), ('``', '.'), ('deserves', 'VERB'), ('the', 'DET'), ('praise', 'NOUN'), ('and', 'CONJ'), ('thanks', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('City

### Separating tags and words in train data


In [220]:
import numpy as np
import pandas as pd

## Firstly creating a Vocabulary out off the training dataset:

### Vocab is nothing but the list of words with frequency more than 1 and which doesn't consist of '/n'

In [235]:
train_words=[]
for i in train_brown:
    for k in i:
        train_words.append(k[0])
    train_words.append('-new-')

In [236]:
#Creating defaultdict of type int for frequencies
frequency = defaultdict(int)

for wd in train_words:
    frequency[wd] += 1



In [264]:
# creating vocab out off the freq
vocab=[]
for key,value in frequency.items():
    if value>1 and key!='\n':
        vocab.append(key)
    
        
vocab=sorted(vocab)

voc = {}
count=0
for i in vocab:
    voc[i]=count
    count+=1
for j in ['-unk-','-unk-WB','-unk-V','-unk-N','-unk-JJ','-unk-dig-','-unk-up']:    
    voc[j] = count
    count+=1

#### Creating a utility function to assign a token to an unkown word , that doesnot exist in the vocabulary

In [277]:
def util_unk(wd):
    
    #Refernce to suffixes for all the part of speeches: http://www.uefap.com/vocab/build/building.htm
    
    adjectives_suff = ['ent','ive','ous','ful','less','able']
    noun_suff = ['tion','sion','er','ment','ant','ent','age','ence','ance','ery','ry','er','ism','ship','age','ity','ness','cy']
    adverb_suff = ['wards','wise','ward']
    verbs_suff = ['ise','ate','fy','en','ize']
    
    if any(c.isdigit() for c in wd):
        return "-unk-dig-"
    elif any(c.isupper() for c in wd):
        return "-unk-up"
    elif any(wd.endswith(suff) for suff in adjectives_suff):
        return "-unk-JJ"
    elif any(wd.endswith(suff) for suff in noun_suff):
        return  "-unk-N"
    elif any(wd.endswith(suff) for suff in verbs_suff):
        return "-unk-V"
    elif any(wd.endswith(suff) for suff in adverb_suff):
        return "-unk-WB"
    
    return "-unk-"
    

#### function for checking vocab and handling unkown tags

In [278]:
def assign_tag(voc,line):
    #handling blank lines
    wd,tag = line[0],line[1]
    if wd not in voc.keys():
        wd = util_unk(wd)
    return wd,tag

#### Processing  above function on train data

In [279]:
# preprocessing train data takes around 9 - 10 min

import time
start = time.time()
prep_train_brown = []
count=0
for i in train_brown:
    for j in i:
        wd,tag = assign_tag(voc,j)
        prep_train_brown.append((wd,tag))
        count+=1
        if count%100000==0:
            print('processed ',count)
    prep_train_brown.append(('-new-','-start-'))
        
end = time.time()
print(f'time taken = {end-start}')

processed  100000
processed  200000
processed  300000
processed  400000
processed  500000
processed  600000
processed  700000
processed  800000
time taken = 3.598200559616089


In [280]:
prep_train_brown

[('The', 'DET'),
 ('Fulton', 'NOUN'),
 ('County', 'NOUN'),
 ('Grand', 'ADJ'),
 ('Jury', 'NOUN'),
 ('said', 'VERB'),
 ('Friday', 'NOUN'),
 ('an', 'DET'),
 ('investigation', 'NOUN'),
 ('of', 'ADP'),
 ("Atlanta's", 'NOUN'),
 ('recent', 'ADJ'),
 ('primary', 'NOUN'),
 ('election', 'NOUN'),
 ('produced', 'VERB'),
 ('``', '.'),
 ('no', 'DET'),
 ('evidence', 'NOUN'),
 ("''", '.'),
 ('that', 'ADP'),
 ('any', 'DET'),
 ('irregularities', 'NOUN'),
 ('took', 'VERB'),
 ('place', 'NOUN'),
 ('.', '.'),
 ('-new-', '-start-'),
 ('The', 'DET'),
 ('jury', 'NOUN'),
 ('further', 'ADV'),
 ('said', 'VERB'),
 ('in', 'ADP'),
 ('-unk-', 'NOUN'),
 ('-unk-', 'NOUN'),
 ('that', 'ADP'),
 ('the', 'DET'),
 ('City', 'NOUN'),
 ('Executive', 'ADJ'),
 ('Committee', 'NOUN'),
 (',', '.'),
 ('which', 'DET'),
 ('had', 'VERB'),
 ('over-all', 'ADJ'),
 ('charge', 'NOUN'),
 ('of', 'ADP'),
 ('the', 'DET'),
 ('election', 'NOUN'),
 (',', '.'),
 ('``', '.'),
 ('deserves', 'VERB'),
 ('the', 'DET'),
 ('praise', 'NOUN'),
 ('and', 'CONJ'

In [281]:
#preprocessing test data takes around 3 min
import time
start = time.time()
prep_test_brown = []
count=0
for i in test_brown:
    for j in i:
        wd,tag = assign_tag(voc,j)
        prep_test_brown.append((wd,tag))
        count+=1
        if count%100000==0:
            print('processed ',count)
    prep_test_brown.append(('-new-','-start-'))
end = time.time()
print(f'time taken = {end-start}')

processed  100000
processed  200000
time taken = 1.1990292072296143


### Train and Test data is preprocessed now.........

In [282]:
prep_test_words = []
prep_test_tags = []
for j in prep_test_brown:
        if len(j) == 2:
            prep_test_words.append(j[0])
            prep_test_tags.append(j[1])


### Creating Tranisition and emission counts dict

In [283]:
def all_dict(voc,train_data):
    transition_cnt = defaultdict(int)
    emission_cnt = defaultdict(int)
    tag_cnt = defaultdict(int)
    
    previous_tag = '-start-'
    
    j=0
    
    for line in train_data:
        j+=1
        wd,tag = assign_tag(voc, line)
        
        transition_cnt[(previous_tag,tag)]+=1
        emission_cnt[(tag,wd)]+=1
        tag_cnt[tag]+=1
        previous_tag = tag
        
        if (j%100000 == 0):
            print('lines processed : {}'.format(j))
        
    return transition_cnt, emission_cnt, tag_cnt    
        

In [284]:
# Execution time ~ 11 min
start = time.time()
transition_cnt, emission_cnt, tag_cnt = all_dict(voc, prep_train_brown)
end = time.time()
print(end-start)

lines processed : 100000
lines processed : 200000
lines processed : 300000
lines processed : 400000
lines processed : 500000
lines processed : 600000
lines processed : 700000
lines processed : 800000
lines processed : 900000
1.3690216541290283


In [285]:
sorted_states = sorted(tag_cnt.keys())


In [286]:
print(sorted_states)

['-start-', '.', 'ADJ', 'ADP', 'ADV', 'CONJ', 'DET', 'NOUN', 'NUM', 'PRON', 'PRT', 'VERB', 'X']


### Calculating Transition and Emission probababilities matrix

#### We will use smoothing parameter = 0.001 to prevent division by zero

In [287]:
# Transition prob matrix
def transition_matrix(transition_cnt,tag_cnt,smooth,sorted_states):
    total_tags = len(tag_cnt)
    
    #numpy transition matrix (array) 'T'
    T = np.zeros((total_tags,total_tags))
    keys_transition = set(transition_cnt.keys())
    for i in range(total_tags):
        for j in range(total_tags):
            cnt = 0
            key = (sorted_states[i],sorted_states[j])
            if key in keys_transition:
                cnt = transition_cnt.get(key)
            prev_tag_cnt = tag_cnt[key[0]]
            
            T[i,j] = ((cnt + smooth)/(prev_tag_cnt + (smooth*total_tags)))
    return T

In [288]:
smooth = 0.001
T = transition_matrix(transition_cnt,tag_cnt,smooth,sorted_states)
T.shape

(13, 13)

In [289]:
# T_sub = pd.DataFrame(T[0:5,0:5], index=sorted_states[0:5], columns = sorted_states[0:5] )
# print(T_sub)

In [290]:
# Emission prob matrix
def emission_matrix(emission_cnt, tag_cnt, smooth, sorted_states,voc):
    total_tags = len(tag_cnt)
    unique_wds = len(voc)
    
    E = np.zeros((total_tags,unique_wds))
    keys_emission = set(list(emission_cnt.keys()))
    for i in range(total_tags):
        for j in range(unique_wds):
            cnt = 0
            key = (sorted_states[i], voc[j])
            if key in emission_cnt.keys():
                cnt = emission_cnt[key]
            prev_tag_cnt = tag_cnt[key[0]]
            
            E[i,j] =  (smooth + cnt)/(unique_wds*smooth + prev_tag_cnt)
    return E

In [291]:
E = emission_matrix(emission_cnt, tag_cnt, smooth, sorted_states, list(voc))

print(f"View Matrix position at row 0, column 0: {E[0,0]:.9f}")
print(f"View Matrix position at row 3, column 1: {E[3,1]:.9f}")


View Matrix position at row 0, column 0: 0.000000025
View Matrix position at row 3, column 1: 0.000000009


### Viterbi Algorithm

#### Let's do this in 3 parts:
1. Initialization
2. Feed Forward
3. Feed Backward


#### Initialization

initialize `best_prob` and `best_path` matrices

In [292]:

def initialization(T,E,sorted_states, tag_cnt, prep, voc):
    total_tags = len(tag_cnt)
    best_prob = np.zeros((total_tags, len(prep)))
    best_path = np.zeros((total_tags, len(prep)),dtype = int)
    initial_ind = sorted_states.index('-start-')
    for i in range(total_tags):
        if T[initial_ind,i] == 0:
            best_prob[i,0] = -(math.inf)
        else:
            best_prob[i,0] = math.log(T[initial_ind,i]) +math.log(E[i,voc[prep[0]]])
    return best_prob,best_path
        

In [293]:
best_prob,best_path = initialization(T,E,sorted_states,tag_cnt, prep_test_words, voc)

#### Feed Forward
Populate `best prob` and `best_path`

In [297]:
def vit_forward(T, E, prep, best_prob, best_path, voc):
    total_tags = best_prob.shape[0]
    for x in range(1,len(prep)):
        if x % 100000 == 0:
            print(f'words processed {x}')
        for y in range(total_tags):
            best_prob_for_x = -(math.inf)
            best_path_for_x = None
            for z in range(total_tags):
#                 print(x,y,z)
#                 print(best_prob[z,x-1])
#                 print(math.log(T[z,y]))
#                 print(math.log(E[y,voc[prep[x]]]))
                new_prob = best_prob[z,x-1] + math.log(T[z,y]) + math.log(E[y,voc[prep[x]]])
                if new_prob > best_prob_for_x:
                    best_prob_for_x = new_prob
                    best_path_for_x = z
            best_prob[y,x] = best_prob_for_x
            best_path[y,x] = best_path_for_x
    return best_prob, best_path
        

In [298]:
best_prob, best_path = vit_forward(T, E, prep_test_words, best_prob, best_path,voc)

words processed 10000
words processed 20000
words processed 30000
words processed 40000
words processed 50000
words processed 60000
words processed 70000
words processed 80000
words processed 90000
words processed 100000
words processed 110000
words processed 120000
words processed 130000
words processed 140000
words processed 150000
words processed 160000
words processed 170000
words processed 180000
words processed 190000
words processed 200000
words processed 210000
words processed 220000
words processed 230000
words processed 240000
words processed 250000
words processed 260000
words processed 270000
words processed 280000


#### Viterbi backward (Backpropagation)

In [308]:
def vit_back(best_path, best_prob, prep, sorted_states):
    total_words = best_path.shape[1]
    x = []
    prediction = []
    for i in range(total_words):
        x.append(None)
        prediction.append(None)
    total_tags = best_prob.shape[0]
    last_word_best_prob = -(math.inf)
    for i in range(total_tags):
        if best_prob[i,total_words-1] > last_word_best_prob:
            last_word_best_prob = best_prob[i,total_words-1]
            x[total_words-1] = i
    prediction[total_words-1] = str(sorted_states[x[total_words-1]])
    for j in range(total_words-1, -1,-1):
        tag_j = x[j]
        x[j-1] = best_path[tag_j,j]
        prediction[j-1] = str(sorted_states[x[j-1]])
    return prediction

In [311]:
prediction = vit_back(best_path, best_prob, prep_test_words, sorted_states)
m= len(prediction)
print('The prediction for pred[-7:m-1] is: \n', prep_test_words[-7:m-1], "\n", prediction[-7:m-1],"\n", prep_test_tags[-7:m-1] ,"\n")
print('The prediction for pred[0:8] is: \n', prep_test_words[0:7], "\n", prediction[0:7], "\n", prep_test_tags[0:7])

The prediction for pred[-7:m-1] is: 
 ['-unk-', '-unk-', 'dress', 'was', '-unk-', '.'] 
 ['ADJ', 'NOUN', 'NOUN', 'VERB', 'NOUN', '.'] 
 ['ADJ', 'NOUN', 'NOUN', 'VERB', 'VERB', '.'] 

The prediction for pred[0:8] is: 
 ['She', 'looked', 'at', 'him', 'in', 'surprise', '.'] 
 ['PRON', 'VERB', 'ADP', 'PRON', 'ADP', 'NOUN', '.'] 
 ['PRON', 'VERB', 'ADP', 'PRON', 'ADP', 'NOUN', '.']


In [320]:
def total_accuracy(prediction, prep_test_tags):
    correct = 0
    total = 0
    
    for pred, actual in zip(prediction, prep_test_tags):
        if actual==pred:
            correct+=1
        total+=1
    return correct/total

In [321]:
print(f"Total Accuracy of Viterbi Algo is {total_accuracy(prediction, prep_test_tags)}")

Total Accuracy of Viterbi Algo is 0.9545506682191829


In [341]:
def per_postag_accuracy(sorted_states,prediction, prep_test_tags):    
    per_pos_tag = {}
    total_per_pos_tag = {}
    per_pos_tag_accuracy = {}
    for i in sorted_states:
        per_pos_tag[i] = 0
        total_per_pos_tag[i] = 0
        per_pos_tag_accuracy[i]=0
    for pred,actual in zip (prediction,prep_test_tags):
        if pred==actual:
            per_pos_tag[pred]+=1
        total_per_pos_tag[actual]+=1
    for i in sorted_states:
        per_pos_tag_accuracy[i]= (per_pos_tag[i]/total_per_pos_tag[i])
    return per_pos_tag_accuracy

In [342]:
per_pos_tag_accuracy = per_postag_accuracy(sorted_states, prediction, prep_test_tags)


In [344]:
per_pos_tag_accuracy

{'-start-': 1.0,
 '.': 0.9998589264301333,
 'ADJ': 0.8803460545594084,
 'ADP': 0.9540221562522627,
 'ADV': 0.8551782571856873,
 'CONJ': 0.9956266543906088,
 'DET': 0.9769261503629221,
 'NOUN': 0.9530121422715028,
 'NUM': 0.9560752994865944,
 'PRON': 0.9831009508794403,
 'PRT': 0.851996007984032,
 'VERB': 0.9445695514740822,
 'X': 0.44841269841269843}